# 🔗 Runnables e LCEL: O Poder da Orquestração no LangChain! 

**Módulo 3 - LangChain v0.3 com Pedro Guth**

Eaí pessoal! Tudo certo? Hoje vamos mergulhar no coração do LangChain: os **Runnables** e a **LCEL (LangChain Expression Language)**!

Lembra do módulo anterior onde mexemos com ChatModels? Agora vamos ver como conectar essas peças como se fosse um LEGO super inteligente!

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-03_img_01.png)

## 🤔 Tá, mas o que são Runnables?

Imagina que você tem uma fábrica de brigadeiros. Você tem:
- A estação que mistura o chocolate 🍫
- A estação que enrola as bolinhas 🥄
- A estação que põe o granulado ✨

Cada estação é um **Runnable** - uma unidade que recebe algo, processa e passa adiante!

No LangChain, **TUDO** é um Runnable:
- ChatModels (que já vimos!)
- PromptTemplates (que vamos ver no próximo módulo!)
- OutputParsers
- Chains
- E por aí vai...

**Dica!** Runnable é a interface padrão que permite conectar qualquer componente do LangChain de forma consistente!

In [ ]:
# Primeiro, vamos instalar e importar o que precisamos
!pip install langchain langchain-google-genai python-dotenv

import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

# Carrega as variáveis de ambiente
load_dotenv()

print("📦 Bibliotecas importadas com sucesso!")

In [ ]:
# Configurando nossa API key do Google (lembra do módulo 2?)
# Substitua por sua chave ou configure no .env
os.environ["GOOGLE_API_KEY"] = "sua_chave_aqui"

# Criando nosso ChatModel (já conhecemos esse cara!)
modelo = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.7
)

print("🤖 Modelo configurado e pronto para usar!")
print(f"Tipo do modelo: {type(modelo)}")
print(f"É um Runnable? {hasattr(modelo, 'invoke')}")

## 🏃‍♂️ Os Métodos Fundamentais dos Runnables

Todo Runnable tem 4 métodos principais:

| Método | O que faz | Quando usar |
|--------|-----------|-------------|
| `invoke()` | Executa uma vez | Quando você quer uma resposta simples |
| `batch()` | Executa em lote | Quando tem várias perguntas de uma vez |
| `stream()` | Executa com streaming | Quando quer ver a resposta chegando aos poucos |
| `ainvoke()` | Executa de forma assíncrona | Para aplicações mais avançadas |

É como ter 4 formas diferentes de pedir um Uber! 🚗

In [ ]:
# Testando o método invoke() - o mais básico
resposta = modelo.invoke("Explique o que é inteligência artificial em uma frase")
print("🎯 Método invoke():")
print(resposta.content)
print("\n" + "="*50 + "\n")

# Testando o método batch() - várias perguntas de uma vez
perguntas = [
    "O que é Python?",
    "O que é JavaScript?",
    "O que é LangChain?"
]

respostas = modelo.batch(perguntas)
print("📦 Método batch():")
for i, resposta in enumerate(respostas):
    print(f"{i+1}. {resposta.content[:100]}...")

In [ ]:
# Testando o método stream() - vendo a resposta chegar aos poucos
print("🌊 Método stream():")
pergunta = "Conte uma história curta sobre um robô que aprende a fazer café"

for chunk in modelo.stream(pergunta):
    print(chunk.content, end="", flush=True)

print("\n\n✨ Liiindo! A resposta chegou aos poucos!")

## ⚡ LCEL - LangChain Expression Language

Agora vem a parte mais massa! A **LCEL** é como se fosse a "linguagem" para conectar Runnables.

É tipo aqueles conectores de mangueira de jardim - você vai encaixando um no outro até formar o sistema perfeito! 🌱

A sintaxe básica é usar o operador `|` (pipe):

```python
chain = componente1 | componente2 | componente3
```

**Dica!** O pipe (`|`) é como uma seta que diz "pega o resultado daqui e manda pra lá"!

```mermaid
graph LR
    A[Input] --> B[Runnable 1]
    B --> C[Runnable 2]
    C --> D[Runnable 3]
    D --> E[Output]
    
    style A fill:#e1f5fe
    style E fill:#c8e6c9
    style B fill:#fff3e0
    style C fill:#fff3e0
    style D fill:#fff3e0
```

In [ ]:
# Vamos criar nossa primeira chain simples!
# Lembra: ChatModel retorna um objeto AIMessage, mas queremos só o texto

# Criando um parser para extrair só o texto
parser = StrOutputParser()

# Nossa primeira chain! 🎉
chain_simples = modelo | parser

print("🔗 Chain criada!")
print(f"Tipo da chain: {type(chain_simples)}")

# Testando nossa chain
resultado = chain_simples.invoke("Me conte uma piada sobre programadores")
print("\n😄 Resultado da chain:")
print(resultado)
print(f"\nTipo do resultado: {type(resultado)}")

In [ ]:
# Vamos criar funções customizadas como Runnables!
# Isso é útil para adicionar lógica personalizada

def adicionar_emoji(texto):
    """Adiciona emojis aleatórios ao texto"""
    emojis = ["🚀", "⭐", "🎯", "💡", "🔥", "✨"]
    import random
    emoji_escolhido = random.choice(emojis)
    return f"{emoji_escolhido} {texto} {emoji_escolhido}"

def deixar_maiusculo(texto):
    """Transforma o texto em maiúsculo"""
    return texto.upper()

# Transformando funções em Runnables
runnable_emoji = RunnableLambda(adicionar_emoji)
runnable_maiusculo = RunnableLambda(deixar_maiusculo)

print("🛠️ Runnables customizados criados!")

In [ ]:
# Agora vamos criar uma chain mais complexa!
# modelo -> parser -> emoji -> maiúsculo

chain_complexa = modelo | parser | runnable_emoji | runnable_maiusculo

resultado = chain_complexa.invoke("Diga algo motivacional sobre aprender IA")

print("🎪 Chain complexa em ação:")
print(resultado)

print("\n🔍 Vamos ver o que aconteceu em cada etapa:")
# Executando passo a passo para entender
etapa1 = modelo.invoke("Diga algo motivacional sobre aprender IA")
print(f"1. Modelo: {etapa1.content[:50]}...")

etapa2 = parser.invoke(etapa1)
print(f"2. Parser: {etapa2[:50]}...")

etapa3 = runnable_emoji.invoke(etapa2)
print(f"3. Emoji: {etapa3[:50]}...")

etapa4 = runnable_maiusculo.invoke(etapa3)
print(f"4. Maiúsculo: {etapa4[:50]}...")

## 🔀 RunnablePassthrough - O Cara que Não Muda Nada

Às vezes você quer que um valor passe pela chain sem ser modificado. É como ter um "atalho" na nossa fábrica de brigadeiros.

O `RunnablePassthrough` é perfeito para isso! Ele recebe algo e passa adiante sem modificar.

**Dica!** Muito útil quando você quer manter o input original em chains mais complexas!

In [ ]:
# Testando o RunnablePassthrough
passthrough = RunnablePassthrough()

# Criando uma chain que preserva o input original
texto_original = "Esta mensagem vai passar inalterada"
resultado = passthrough.invoke(texto_original)

print("🔄 RunnablePassthrough em ação:")
print(f"Input: {texto_original}")
print(f"Output: {resultado}")
print(f"São iguais? {texto_original == resultado}")

# Exemplo prático: chain que retorna tanto o original quanto processado
def processar_texto(texto):
    return f"Processado: {texto.upper()}"

runnable_processar = RunnableLambda(processar_texto)

# Chain que mostra antes e depois
texto_teste = "olá mundo"
original = passthrough.invoke(texto_teste)
processado = runnable_processar.invoke(texto_teste)

print(f"\n📊 Comparação:")
print(f"Original: {original}")
print(f"Processado: {processado}")

## 🎭 Chains Paralelas - Fazendo Várias Coisas ao Mesmo Tempo

E se você quiser fazer várias operações em paralelo? É como ter várias linhas de produção na fábrica!

No LangChain, usamos dicionários para criar execução paralela:

```python
chain_paralela = {
    "operacao1": runnable1,
    "operacao2": runnable2
}
```

In [ ]:
# Criando funções para processar texto de formas diferentes
def contar_palavras(texto):
    """Conta as palavras no texto"""
    palavras = len(texto.split())
    return f"Tem {palavras} palavras"

def contar_caracteres(texto):
    """Conta os caracteres no texto"""
    chars = len(texto)
    return f"Tem {chars} caracteres"

def primeira_palavra(texto):
    """Pega a primeira palavra"""
    primeira = texto.split()[0] if texto.split() else "Nenhuma"
    return f"Primeira palavra: {primeira}"

# Transformando em Runnables
runnable_palavras = RunnableLambda(contar_palavras)
runnable_chars = RunnableLambda(contar_caracteres)
runnable_primeira = RunnableLambda(primeira_palavra)

print("⚡ Runnables para análise de texto criados!")

In [ ]:
# Criando uma chain paralela!
# Primeiro, vamos fazer o modelo gerar um texto
# Depois, vamos analisar esse texto de 3 formas diferentes

chain_paralela = modelo | parser | {
    "palavras": runnable_palavras,
    "caracteres": runnable_chars,
    "primeira": runnable_primeira,
    "original": RunnablePassthrough()
}

resultado = chain_paralela.invoke("Escreva uma frase sobre o futuro da tecnologia")

print("🔀 Chain paralela em ação:")
print(f"Original: {resultado['original']}")
print(f"Análise - {resultado['palavras']}")
print(f"Análise - {resultado['caracteres']}")
print(f"Análise - {resultado['primeira']}")

```mermaid
graph TD
    A[Input] --> B[Modelo]
    B --> C[Parser]
    C --> D[Execução Paralela]
    D --> E[Contar Palavras]
    D --> F[Contar Caracteres]
    D --> G[Primeira Palavra]
    D --> H[Original]
    E --> I[Output Dict]
    F --> I
    G --> I
    H --> I
    
    style A fill:#e1f5fe
    style I fill:#c8e6c9
    style D fill:#fff3e0
```

## 📊 Visualizando o Fluxo das Chains

Bora criar um gráfico para entender melhor como os dados fluem pelas chains!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simulando tempos de execução de diferentes chains
chains = ['Simples\n(Modelo + Parser)', 'Complexa\n(4 etapas)', 'Paralela\n(4 operações)']
tempos = [1.2, 2.1, 1.8]  # Tempos simulados em segundos
cores = ['#FF6B6B', '#4ECDC4', '#45B7D1']

fig, ax = plt.subplots(figsize=(10, 6))
barras = ax.bar(chains, tempos, color=cores, alpha=0.8, edgecolor='white', linewidth=2)

# Adicionando valores nas barras
for i, (barra, tempo) in enumerate(zip(barras, tempos)):
    ax.text(barra.get_x() + barra.get_width()/2, barra.get_height() + 0.05, 
            f'{tempo}s', ha='center', va='bottom', fontweight='bold', fontsize=12)

ax.set_title('⚡ Comparação de Desempenho das Chains', fontsize=16, fontweight='bold', pad=20)
ax.set_ylabel('Tempo de Execução (segundos)', fontsize=12)
ax.set_ylim(0, max(tempos) * 1.3)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# Adicionando anotações
ax.annotate('Mais rápida!', xy=(0, tempos[0]), xytext=(0.5, tempos[0] + 0.5),
            arrowprops=dict(arrowstyle='->', color='green', lw=2),
            fontsize=10, ha='center', color='green', fontweight='bold')

plt.tight_layout()
plt.show()

print("📈 Gráfico criado! Como você pode ver, chains paralelas são eficientes!")

## 🎯 Exemplo Prático: Sistema de Análise de Sentimentos

Vamos criar um sistema real que usa tudo que aprendemos! Vamos analisar o sentimento de textos de forma completa.

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-03_img_03.png)

In [ ]:
# Vamos criar um sistema completo de análise de sentimentos!

def criar_prompt_sentimento(texto):
    """Cria um prompt específico para análise de sentimentos"""
    prompt = f"""
    Analise o sentimento do seguinte texto e retorne apenas uma das opções:
    - POSITIVO
    - NEGATIVO  
    - NEUTRO
    
    Texto: {texto}
    
    Sentimento:
    """
    return prompt

def extrair_sentimento(resposta):
    """Extrai o sentimento da resposta do modelo"""
    resposta = resposta.upper().strip()
    if "POSITIVO" in resposta:
        return "POSITIVO 😊"
    elif "NEGATIVO" in resposta:
        return "NEGATIVO 😢"
    elif "NEUTRO" in resposta:
        return "NEUTRO 😐"
    else:
        return "INDEFINIDO 🤔"

def contar_sentimentos_palavras(texto):
    """Conta palavras que indicam sentimentos"""
    palavras_positivas = ['bom', 'ótimo', 'excelente', 'feliz', 'alegre', 'amor', 'sucesso']
    palavras_negativas = ['ruim', 'péssimo', 'triste', 'raiva', 'ódio', 'fracasso', 'problema']
    
    texto_lower = texto.lower()
    pos = sum(1 for palavra in palavras_positivas if palavra in texto_lower)
    neg = sum(1 for palavra in palavras_negativas if palavra in texto_lower)
    
    return f"Palavras positivas: {pos}, Negativas: {neg}"

# Transformando em Runnables
runnable_prompt = RunnableLambda(criar_prompt_sentimento)
runnable_extrair = RunnableLambda(extrair_sentimento)
runnable_contar = RunnableLambda(contar_sentimentos_palavras)

print("🎭 Sistema de análise de sentimentos criado!")

In [ ]:
# Montando nossa chain completa de análise de sentimentos
chain_sentimentos = {
    "analise_ia": runnable_prompt | modelo | parser | runnable_extrair,
    "analise_palavras": runnable_contar,
    "texto_original": RunnablePassthrough(),
    "tamanho": RunnableLambda(lambda x: f"Caracteres: {len(x)}")
}

# Testando com diferentes textos
textos_teste = [
    "Estou muito feliz com meu progresso em IA! Está sendo uma jornada incrível!",
    "Esse código não funciona e estou muito frustrado com esses erros.",
    "Hoje é segunda-feira e tenho uma reunião às 14h."
]

print("🔍 Analisando diferentes textos:\n")

for i, texto in enumerate(textos_teste, 1):
    print(f"📄 Texto {i}: {texto[:50]}...")
    resultado = chain_sentimentos.invoke(texto)
    
    print(f"   🤖 IA diz: {resultado['analise_ia']}")
    print(f"   📊 Contagem: {resultado['analise_palavras']}")
    print(f"   📏 {resultado['tamanho']}")
    print("-" * 60)

## 🚀 Streaming com Chains - Vendo em Tempo Real

Uma das coisas mais legais das chains é que você pode fazer streaming! É como ver a mágica acontecer em tempo real.

**Dica!** Streaming é especialmente útil para aplicações web onde você quer mostrar a resposta chegando aos poucos!

In [ ]:
# Vamos criar uma chain simples para streaming
chain_streaming = modelo | parser

print("🌊 Demonstração de streaming:")
print("Pergunta: 'Explique como funciona o machine learning'")
print("Resposta chegando aos poucos...\n")
print("-" * 50)

# Fazendo streaming da resposta
for chunk in chain_streaming.stream("Explique como funciona o machine learning em termos simples"):
    print(chunk, end="", flush=True)

print("\n" + "-" * 50)
print("\n✨ Streaming finalizado! Legal né?")

## 🎮 Exercício Prático 1: Sua Primeira Chain!

Agora é sua vez! Vamos criar uma chain que:
1. Recebe um tópico
2. Pede para o modelo criar uma pergunta sobre esse tópico
3. Conta quantas palavras tem a pergunta
4. Adiciona emojis relacionados ao tópico

**Desafio:** Complete o código abaixo!

In [ ]:
# 🎯 EXERCÍCIO 1: Complete a chain abaixo

def criar_pergunta(topico):
    """Cria um prompt para gerar uma pergunta sobre o tópico"""
    # TODO: Complete esta função
    # Dica: retorne um prompt pedindo uma pergunta interessante sobre o tópico
    pass

def adicionar_emoji_topico(texto):
    """Adiciona emojis baseados no conteúdo"""
    # TODO: Complete esta função
    # Dica: use if/elif para diferentes tópicos (tecnologia, comida, esporte, etc.)
    pass

def contar_info(texto):
    """Conta palavras e caracteres"""
    # TODO: Complete esta função
    # Dica: retorne algo como "X palavras, Y caracteres"
    pass

# Transforme suas funções em Runnables
# TODO: Crie os RunnableLambda

# Crie a chain completa
# TODO: Monte a chain usando o operador |

# Teste com um tópico
# resultado = sua_chain.invoke("inteligência artificial")
# print(resultado)

## 💡 Solução do Exercício 1

Aqui está uma possível solução para o exercício anterior:

In [ ]:
# ✅ SOLUÇÃO DO EXERCÍCIO 1

def criar_pergunta(topico):
    """Cria um prompt para gerar uma pergunta sobre o tópico"""
    return f"Crie uma pergunta interessante e educativa sobre: {topico}"

def adicionar_emoji_topico(texto):
    """Adiciona emojis baseados no conteúdo"""
    texto_lower = texto.lower()
    
    if any(palavra in texto_lower for palavra in ['tecnologia', 'ia', 'computador', 'software']):
        return f"💻 {texto} 🤖"
    elif any(palavra in texto_lower for palavra in ['comida', 'receita', 'cozinha']):
        return f"🍳 {texto} 🍽️"
    elif any(palavra in texto_lower for palavra in ['esporte', 'futebol', 'basquete']):
        return f"⚽ {texto} 🏆"
    else:
        return f"📚 {texto} 🎓"

def contar_info(texto):
    """Conta palavras e caracteres"""
    palavras = len(texto.split())
    caracteres = len(texto)
    return f"{texto} | [Info: {palavras} palavras, {caracteres} caracteres]"

# Criando os Runnables
runnable_pergunta = RunnableLambda(criar_pergunta)
runnable_emoji_topico = RunnableLambda(adicionar_emoji_topico)
runnable_info = RunnableLambda(contar_info)

# Chain completa
chain_exercicio = runnable_pergunta | modelo | parser | runnable_emoji_topico | runnable_info

# Testando
resultado = chain_exercicio.invoke("inteligência artificial")
print("🎯 Resultado do exercício:")
print(resultado)

## 🎲 Exercício Prático 2: Chain Paralela Avançada

Agora vamos criar algo mais desafiador! Uma chain que processa um texto de múltiplas formas simultaneamente.

**Objetivo:** Criar uma chain que analise um texto e retorne:
- Resumo em uma frase
- Tradução para inglês  
- Análise de sentimento
- Estatísticas básicas

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-03_img_04.png)

In [ ]:
# 🎯 EXERCÍCIO 2: Chain Paralela Avançada

# Funções auxiliares já prontas
def criar_prompt_resumo(texto):
    return f"Resuma este texto em uma frase: {texto}"

def criar_prompt_traducao(texto):
    return f"Traduza este texto para o inglês: {texto}"

def criar_prompt_sentimento_simples(texto):
    return f"Qual o sentimento deste texto? Responda apenas: POSITIVO, NEGATIVO ou NEUTRO. Texto: {texto}"

def estatisticas_texto(texto):
    palavras = len(texto.split())
    frases = texto.count('.') + texto.count('!') + texto.count('?')
    chars = len(texto)
    return f"Estatísticas: {palavras} palavras, {frases} frases, {chars} caracteres"

# TODO: Crie os Runnables necessários
# TODO: Monte a chain paralela usando dicionário
# TODO: Teste com um texto interessante

# Dica: Lembre-se que para chains que usam o modelo, você precisa:
# prompt_function | modelo | parser

texto_teste = """
A inteligência artificial está revolucionando a forma como trabalhamos e vivemos. 
Com o LangChain, conseguimos criar aplicações incríveis que antes pareciam impossíveis. 
É uma época fantástica para ser um desenvolvedor!
"""

print("📝 Texto para análise:")
print(texto_teste)
print("\n" + "="*60 + "\n")

# TODO: Execute sua chain aqui
# resultado = sua_chain_paralela.invoke(texto_teste)
# print(resultado)

In [ ]:
# ✅ SOLUÇÃO DO EXERCÍCIO 2

# Criando os Runnables
runnable_prompt_resumo = RunnableLambda(criar_prompt_resumo)
runnable_prompt_traducao = RunnableLambda(criar_prompt_traducao)
runnable_prompt_sentimento = RunnableLambda(criar_prompt_sentimento_simples)
runnable_estatisticas = RunnableLambda(estatisticas_texto)

# Chain paralela completa
chain_analise_completa = {
    "resumo": runnable_prompt_resumo | modelo | parser,
    "traducao": runnable_prompt_traducao | modelo | parser,
    "sentimento": runnable_prompt_sentimento | modelo | parser,
    "estatisticas": runnable_estatisticas,
    "original": RunnablePassthrough()
}

# Executando a análise
resultado = chain_analise_completa.invoke(texto_teste)

print("🔍 ANÁLISE COMPLETA DO TEXTO:")
print("="*60)
print(f"📊 {resultado['estatisticas']}")
print(f"📝 Resumo: {resultado['resumo']}")
print(f"🌍 Tradução: {resultado['traducao']}")
print(f"🎭 Sentimento: {resultado['sentimento']}")
print("="*60)
print("\n🎉 Liiindo! Conseguimos analisar o texto de 4 formas diferentes simultaneamente!")

## 📈 Visualizando a Performance das Chains

Vamos criar um gráfico para comparar a eficiência das diferentes abordagens que aprendemos!

In [ ]:
import matplotlib.pyplot as plt
import time
import numpy as np

# Simulando tempos de execução para diferentes tipos de chains
tipos_chain = ['Sequencial\nSimples', 'Sequencial\nComplexa', 'Paralela\n4 operações', 'Streaming\nTempo Real']
tempos_medio = [0.8, 2.5, 1.2, 0.9]
complexidade = [1, 4, 4, 2]  # Número de operações

# Criando subplot com dois gráficos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Tempo de Execução
cores1 = ['#FF6B6B', '#FF8E53', '#FF6B9D', '#4ECDC4']
barras1 = ax1.bar(tipos_chain, tempos_medio, color=cores1, alpha=0.8, edgecolor='white', linewidth=2)

for barra, tempo in zip(barras1, tempos_medio):
    ax1.text(barra.get_x() + barra.get_width()/2, barra.get_height() + 0.05, 
             f'{tempo}s', ha='center', va='bottom', fontweight='bold')

ax1.set_title('⚡ Tempo de Execução por Tipo de Chain', fontsize=14, fontweight='bold')
ax1.set_ylabel('Tempo (segundos)')
ax1.grid(axis='y', alpha=0.3)

# Gráfico 2: Complexidade vs Eficiência
scatter = ax2.scatter(complexidade, tempos_medio, c=cores1, s=200, alpha=0.7, edgecolors='white', linewidth=2)

for i, tipo in enumerate(tipos_chain):
    ax2.annotate(tipo.replace('\n', ' '), (complexidade[i], tempos_medio[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=9)

ax2.set_title('🎯 Complexidade vs Tempo de Execução', fontsize=14, fontweight='bold')
ax2.set_xlabel('Número de Operações')
ax2.set_ylabel('Tempo (segundos)')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Observações importantes:")
print("• Chains paralelas são mais eficientes que sequenciais complexas")
print("• Streaming oferece boa experiência do usuário")
print("• Complexidade nem sempre significa maior tempo de execução")

## 🔮 Conectando com os Próximos Módulos

Agora que você domina Runnables e LCEL, vamos ver como eles se conectam com o resto do curso:

### 📋 Módulo 4 - Prompt Templates
- Os PromptTemplates são Runnables!
- Você vai usar LCEL para conectar templates com modelos
- Exemplo: `template | modelo | parser`

### 🎯 Módulo 5 - OutputParsers
- OutputParsers também são Runnables!
- Você vai criar chains do tipo: `modelo | custom_parser`

### ⛓️ Módulo 6 - Chains
- Chains mais complexas usando tudo que aprendemos aqui
- Composição de múltiplos Runnables

**Dica!** Tudo no LangChain é um Runnable - esse é o segredo para criar aplicações poderosas!

## 🎓 Resumo do Módulo - O que Aprendemos?

Bora recapitular tudo que vimos neste módulo:

### ✅ Conceitos Fundamentais
- **Runnables**: A interface padrão de tudo no LangChain
- **LCEL**: A linguagem para conectar componentes
- **Operador Pipe (`|`)**: Para conectar Runnables

### ✅ Métodos dos Runnables
- `invoke()`: Execução simples
- `batch()`: Execução em lote
- `stream()`: Execução com streaming
- `ainvoke()`: Execução assíncrona

### ✅ Tipos de Chains
- **Sequenciais**: Uma operação após a outra
- **Paralelas**: Múltiplas operações simultâneas
- **Complexas**: Combinação de ambas

### ✅ Componentes Especiais
- `RunnableLambda`: Para funções customizadas
- `RunnablePassthrough`: Para manter dados inalterados
- `StrOutputParser`: Para extrair texto simples

### 🚀 Próximos Passos
No próximo módulo vamos aprender sobre **Prompt Templates** - como criar prompts dinâmicos e reutilizáveis que se conectam perfeitamente com as chains que criamos hoje!

**Bora continuar essa jornada incrível! 🎉**

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-03_img_05.png)